In [1]:
import os
import pandas as pd
import ast
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
dacs_dataset_dir = './outputs/hiratamura_dacs_dataset'

PROMPT_PATH = './prompts/prompt.txt'
GPT_MODEL = 'gpt-4o'

In [4]:
# open meta.xlsx
with open(os.path.join(dacs_dataset_dir, 'meta.xlsx'), 'rb') as f:
    root_ja_df = pd.read_excel(f, sheet_name='root_ja')
    files_ja_df = pd.read_excel(f, sheet_name='files_ja')

In [5]:
root_ja_df.iloc[0]

data_id                  i1_bridge_excel_0
title                          蕨平ボックスカルバート
summary                                NaN
point           ("37.174170","140.595280")
archive_file                           NaN
Name: 0, dtype: object

In [6]:
files_ja_df.iloc[0]

data_id                       i1_bridge_excel_0
file_id    13e37566-60f9-11ef-871a-a08069f5749f
file               37.17417,140.59528_2020.xlsx
Name: 0, dtype: object

- files_jaに記された77条excelをもとに健全性の判定区分を取得し、root_jaのsummaryを更新

In [24]:
def read_77_excel_as_text(file_path):
    df = pd.read_excel(file_path, sheet_name=0, engine='openpyxl')  # read the first sheet

    text = ''
    for index, row in df.iterrows():
        for col_name in df.columns:
            cell_value = row[col_name]
            if not pd.isna(cell_value):  # NaNでない値のみを処理
                text += str(cell_value) + '\n'
        text += '\n'
    return text

def get_soundness(text, prompt, gpt_model=GPT_MODEL):  # 健全性判定
    user_prompt = prompt + text
    messages = [
        {'role': 'system', 'content': 'You are a bridge structural engineer.'},
        {'role': 'user', 'content': user_prompt}
    ]

    client = OpenAI()
    completion = client.chat.completions.create(
        model=gpt_model,
        response_format={"type":"json_object"}, 
        messages=messages,
    )
    # print(completion.choices[0].message.content)
    j = ast.literal_eval(completion.choices[0].message.content)
    
    return j['hantei_kubun']

In [25]:
with open(PROMPT_PATH, 'r') as f:
    prompt = f.read()

for i in range(files_ja_df.shape[0]):
    data_id = files_ja_df.iloc[i]['data_id']
    file = files_ja_df.iloc[i]['file']
    file_path = os.path.join(dacs_dataset_dir, data_id, file)

    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
        continue

    text = read_77_excel_as_text(file_path)
    for j in range(10): # gptがエラーを返すことがあるのでリトライ
        try:
            soundness = get_soundness(text, prompt)
            break
        except Exception as e:
            print(f'Error: {e}')
            soundness = 0
    else:
        print(f'Failed to get soundness: {file_path}')

    # arabic to roman number
    num_dict = {
        '1': 'I',
        '2': 'II',
        '3': 'III',
        '4': 'IV',
    }
    if soundness in range(1, 5):
        soundness = num_dict[str(soundness)]
        root_ja_df.loc[root_ja_df['data_id'] == data_id, 'summary'] = f'判定区分: {soundness}'
    else:
        root_ja_df.loc[root_ja_df['data_id'] == data_id]['summary'] = '判定区分: 不明/エラー'

{"hantei_kubun": 3}
{"hantei_kubun": 2 }
{
  "hantei_kubun": 2
}
{"hantei_kubun": 2 }

{
    "hantei_kubun": 2
}


In [27]:
# update meta.xlsx
# 試験用に別ファイル(updated_meta.xlsx)に保存することとする
with pd.ExcelWriter(os.path.join(dacs_dataset_dir, 'updated_meta.xlsx'), engine='openpyxl') as writer:
    root_ja_df.to_excel(writer, sheet_name='root_ja', index=False)
    files_ja_df.to_excel(writer, sheet_name='files_ja', index=False)